In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:


companies = ["NVDA","MSFT","TSLA","AAPL"]
start_dates, end_dates = datetime(2014,1,1),datetime.now()
# Get data

def get_data(company_name:list,start_date:datetime,end_date:datetime):
    stock = yf.download(company_name,start=start_date,end=end_date)
    return stock

nasdaq_5 = get_data(company_name=companies,start_date=start_dates,end_date=end_dates)
nasdaq_5.columns = ["Close","High","Low","Open","Volume"]



In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# 1. Funct to download data
def get_data(company_name:list, start_date:datetime, end_date:datetime):
    return yf.download(company_name, start=start_date, end=end_date)

# 2. Funct to process data of each company
def process_company(df_multi, ticker):
    # Extract only data for each ticker
    df_company = df_multi.xs(ticker, level=1, axis=1).copy()
    
    # Rename columns with of ticker
    df_company.columns = [f"{col}_{ticker}" for col in df_company.columns]
    
    return df_company

# 3.funct to process them all and gather.
def process_all_companies(df_multi, tickers):
    processed = []
    
    for ticker in tickers:
        df = process_company(df_multi, ticker)
        processed.append(df)
        
    # Concat all for index.
    df_final = pd.concat(processed, axis=1)
    return pd.DataFrame(df_final)
# Exect all.
companies = ["NVDA", "MSFT", "TSLA", "AAPL"]
start_dates, end_dates = datetime(2014,1,1), datetime.now()

nasdaq_5 = get_data(company_name=companies, start_date=start_dates, end_date=end_dates)
nasdaq_processed = process_all_companies(nasdaq_5, companies)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  4 of 4 completed


In [3]:
#Daily metrics
#Selecting Close column price
close_nasdaq_5 = nasdaq_processed[["Close_AAPL","Close_MSFT","Close_NVDA","Close_TSLA"]]
#Retorno simples diarios
daily_change = close_nasdaq_5.pct_change(fill_method=None)
acumulate_dreturn = daily_change.mean()*100
acumulate_drisk = daily_change.std()*100
leader = (acumulate_dreturn / acumulate_drisk)*100
data_frame = pd.DataFrame(data=[acumulate_dreturn,acumulate_drisk,leader])
data_frame.columns = companies
data_frame.index = ["Return","Risk","Dominance"]



In [9]:
import plotly.graph_objects as go
import os
import webbrowser
# Crear figura vacía
fig = go.Figure()

# Agregar una línea por cada empresa
for company in companies:
    fig.add_trace(
        go.Scatter(
            x=close_nasdaq_5.index,
            y=close_nasdaq_5[f"Close_{company}"],
            mode='lines',
            name=company
        )
    )

# Personalización del gráfico
fig.update_layout(
    title="Historical Comparison Close price, created by Jose Potes",
    xaxis_title="Date",
    yaxis_title="Close (USD)",
    template="plotly_dark",
    hovermode="x unified",
    
)
fig.add_annotation(
    text=(
        "Creado por Jose Potes | "
        "🔗 LinkedIn: https://www.linkedin.com/in/jose-potes-99280330b/ | 🐦 Twitter: https://x.com/JosePotesFN | 📘 Facebook: https://acortar.link/CzmdlW"
    ),
    xref="paper", yref="paper",
    x=0, y=-0.2,
    showarrow=False,
    font=dict(size=12, color="lightgray"),
    align="center"
)


# Guardar como HTML
output_path = "grafico_nasdaq.html"
fig.write_html(output_path)

# Abrir en navegador automáticamente
webbrowser.open('file://' + os.path.realpath(output_path))

True